In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this step, we mounted Google Drive to our Colab environment. This allows our group to access, save, and load files directly from Google Drive, such as our best-performing model, tokenizers, datasets, and output Excel files. By connecting Drive, we can ensure that all important files are stored persistently and easily shared among team members.

In [2]:
!pip install gradio sacrebleu rouge-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=14296bcd3d7bd35986d5d5d0986d53ba2287ccfa34f58de960e75d060fdca93d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In this step, our group installed Gradio, a Python library that allows us to create an interactive web-based interface for our grammar correction model. With Gradio, we can easily test the model by inputting sentences and instantly viewing the corrected outputs, making it simple to demonstrate and explore the model’s performance in real time.

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Define the path to your pre-trained T5 model on Google Drive
gdrive_model_path = "/content/drive/My Drive/best_model"

gdrive_tokenizer = T5Tokenizer.from_pretrained(gdrive_model_path)
gdrive_model = T5ForConditionalGeneration.from_pretrained(gdrive_model_path)

model = gdrive_model
tokenizer = gdrive_tokenizer

print(f"Successfully loaded T5 tokenizer and model from: {gdrive_model_path}")
print("Model and tokenizer loaded from local path successfully.")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Successfully loaded T5 tokenizer and model from: /content/drive/My Drive/best_model
Model and tokenizer loaded from local path successfully.


In this step, our group loaded the pre-trained T5 grammar correction model and its tokenizer directly from Google Drive. By specifying the path to the saved model, we ensure that the exact trained weights and vocabulary are used for inference. This allows us to run predictions and test the model without retraining. The print statements confirm that both the model and tokenizer were successfully loaded and ready for use.

In [6]:
import sacrebleu
from rouge_score import rouge_scorer

def correct(sentence):
    inputs = tokenizer("grammar: " + sentence, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=128)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

def compute_bleu(original, corrected):
    bleu = sacrebleu.corpus_bleu([corrected], [[original]])
    return round(bleu.score, 2)

def compute_rouge(original, corrected):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(original, corrected)

    result = {
        "ROUGE-1": round(scores["rouge1"].fmeasure, 3),
        "ROUGE-L": round(scores["rougeL"].fmeasure, 3)
    }
    return result


In this step, our group defined functions to use the T5 model for grammar correction and to evaluate its output. The correct function takes an input sentence, prepends a task prefix ("grammar: "), tokenizes it, and uses the model to generate a corrected version.

Additionally, we defined functions to compute evaluation metrics for the model’s output. compute_bleu calculates a BLEU score between the corrected sentence and the original reference, while compute_rouge calculates ROUGE-1 and ROUGE-L scores, providing a structured way to assess the quality of the corrections.

This setup allows us to quickly run the model and measure its performance on individual sentences or datasets.

In [7]:
def correct_file(file):
    text = file.read().decode()
    lines = text.split("\n")
    corrected = [correct(line) for line in lines if line.strip() != ""]
    return "\n".join(corrected)


In this step, our group created a function to apply the grammar correction model to the contents of an uploaded text file. The correct_file function reads the file, splits it into individual lines, applies the correct function to each non-empty line, and then joins the corrected lines back together.

This allows users to upload a text file and automatically receive a fully corrected version, making the model practical for batch corrections rather than just single sentences.

In [13]:
import gradio as gr

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

* {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
}

body {
    background-color: #ffffff;
    color: #1a1a1a;
}

.gradio-container {
    max-width: 900px;
    margin: 0 auto;
}

/* Header styling */
.header-container h1 {
    font-size: 32px;
    font-weight: 700;
    letter-spacing: -0.5px;
    margin-bottom: 8px;
    color: #000000;
}

.header-container p {
    font-size: 14px;
    font-weight: 400;
    letter-spacing: 0.3px;
    color: #666666;
    text-transform: uppercase;
    margin: 0;
}

/* Tabs styling */
.tabs {
    border-bottom: 1px solid #e5e5e5;
}

.tabitem {
    padding: 20px 0;
}

/* Chatbot styling */
.gr-chatbot {
    border: 1px solid #e5e5e5;
    border-radius: 8px;
    background-color: #fafafa;
}

.gr-chatbot textarea {
    border-radius: 6px;
    border: 1px solid #e5e5e5;
    font-size: 14px;
    font-weight: 400;
}

/* Input field styling */
.gr-textbox input {
    border: 1px solid #e5e5e5;
    border-radius: 6px;
    font-size: 14px;
    font-weight: 400;
    padding: 12px;
}

.gr-textbox textarea {
    border: 1px solid #e5e5e5;
    border-radius: 6px;
    font-size: 14px;
    font-weight: 400;
}

/* Button styling */
.gr-button {
    border-radius: 6px;
    font-weight: 500;
    font-size: 14px;
    letter-spacing: 0.3px;
    border: none;
    transition: all 0.2s ease;
}

.gr-button.primary {
    background-color: #000000;
    color: #ffffff;
}

.gr-button.primary:hover {
    background-color: #333333;
}

.gr-button.secondary {
    background-color: #f0f0f0;
    color: #000000;
    border: 1px solid #e5e5e5;
}

.gr-button.secondary:hover {
    background-color: #e5e5e5;
}

/* Labels styling */
.gr-textbox label,
.gr-file label,
.gr-chatbot label {
    font-weight: 600;
    font-size: 13px;
    color: #1a1a1a;
    letter-spacing: 0.2px;
}

/* Placeholder text */
.gr-textbox input::placeholder {
    color: #999999;
    font-weight: 400;
}

/* Row and column spacing */
.gr-row {
    gap: 12px;
}

.gr-column {
    gap: 12px;
}
"""

with gr.Blocks(theme=gr.themes.Monochrome(), css=custom_css) as demo:
    gr.Markdown("""
    <div class='header-container' style='text-align:center; padding: 30px 20px 20px;'>
        <h1 style='color: #ffffff;'>Grammar Checker</h1>
    </div>
    """)

    with gr.Tab("Chatbot"):
        with gr.Column():
            chatbot = gr.Chatbot(label="Conversation", height=500)

        with gr.Row():
            msg = gr.Textbox(
                label="Enter your text",
                placeholder="Type here and press Enter...",
                scale=4
            )
            send_btn = gr.Button("Send", variant="primary", scale=1)

        with gr.Row():
            clear_btn = gr.Button("Clear Chat", scale=1)

        with gr.Row():
            original_box = gr.Textbox(label="Original", lines=2, interactive=False)
            corrected_box = gr.Textbox(label="Corrected", lines=2, interactive=False)

        def chat(history, user_input):
            corrected = correct(user_input)

            history.append((user_input, corrected))
            return (
                history,
                "",
                user_input,
                corrected
            )

        send_btn.click(
            chat,
            [chatbot, msg],
            [chatbot, msg, original_box, corrected_box]
        )

        msg.submit(
            chat,
            [chatbot, msg],
            [chatbot, msg, original_box, corrected_box]
        )

        clear_btn.click(lambda: None, None, chatbot)

    with gr.Tab("Upload File"):
        with gr.Row():
            file_input = gr.File(type="binary", label="Upload .txt")
            file_output = gr.Textbox(label="Corrected Output", lines=15)
        file_btn = gr.Button("Correct File", variant="primary")
        file_btn.click(correct_file, file_input, file_output)

demo.launch()


/tmp/ipython-input-988700233.py:141: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=500)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f055857182469bd6e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In this step, our group built a user-friendly interface using Gradio to make the T5 grammar correction model accessible for anyone, even without coding experience. We created a polished design using custom CSS to style the interface, including fonts, colors, buttons, textboxes, and layout.

The interface has two main tabs:

Chatbot Tab – allows users to type a sentence and get immediate grammar corrections. It displays the conversation history, the original sentence, and the corrected output side by side. Users can send messages by pressing Enter or clicking the Send button, and the chat can be cleared with a button.

Upload File Tab – enables batch corrections by uploading a .txt file. Each line is processed using the correct_file function, and the fully corrected text is displayed in a textbox.

This setup makes the grammar correction model highly interactive, visually appealing, and practical for both single-sentence corrections and large text files, bridging the gap between AI functionality and user experience.